# Introduction

# Warm-up: Find all historical names for a company

# Leverage Industry Codes

## Use SIC codes to evaluate the coverage of record linkages

## SIC codes of interest

## Select a meaningful subset of respondents

### Electricity

### Natural Gas

### Fuel

### Links between companies in different industries

# Leverage Subsidiary Relationships

## Find all historical subsidiaries of a company

## Working with multiple layers of subsidiary nesting